# Simple stock recommendation system with tech indicator - (RSI , MACD) 
       -- by Python and lite SQL

In [17]:
import sqlalchemy
import sqlite3
import ta

import numpy as np 
import pandas as pd 
import datetime as dt 
from pandas_datareader import data as pdr 

In [4]:
engine = sqlalchemy.create_engine('sqlite:///DowJones')

In [12]:
company = pd.read_sql('SELECT name FROM sqlite_master WHERE type = "table"',engine).name.to_list()
company[0:5]

['MMM', 'AXP', 'AMGN', 'AAPL', 'BA']

In [15]:
frame_list = [ pd.read_sql(f'SELECT Date,Close FROM"{name}"',engine) for name in company] 
frame_list[0]

,Date,Close
0,1970-01-02 00:00:00.000000,6.851563
1,1970-01-05 00:00:00.000000,6.890625
2,1970-01-06 00:00:00.000000,6.960938
3,1970-01-07 00:00:00.000000,7.000000
4,1970-01-08 00:00:00.000000,7.093750
...,...,...
12960,2021-05-19 00:00:00.000000,202.600006
12961,2021-05-20 00:00:00.000000,201.649994
12962,2021-05-21 00:00:00.000000,201.860001
12963,2021-05-24 00:00:00.000000,202.610001


In [21]:
def MACD_decision(df):
    df['MACD_diff'] = ta.trend.macd_diff(df.Close)
    df.loc[ (df.MACD_diff>0) & (df.MACD_diff.shift(1) <0 ) ,'Decision MACD'] = 'Buy'

def RSI_decision(df):
    df['RSI'] = ta.momentum.rsi(df.Close,window=10)
    df['SMA200'] = ta.trend.sma_indicator(df.Close,window=200)
    df.loc[ (df.Close > df.SMA200) & (df.RSI < 30) ,'Decision RSI/SMA' ] = 'Buy'


In [22]:
for frame in frame_list : 
    MACD_decision(frame)
    RSI_decision(frame)

In [28]:
output = []
for name,frame in zip(company,frame_list):
    if frame['Decision MACD'].iloc[-1] =='Buy' or frame['Decision RSI/SMA'].iloc[-1] =='Buy':
        print('buying signal for ' + str(name) )
        output.append ( frame )
    else:
        print("Na for " +str(name))

Na for MMM
Na for AXP
Na for AMGN
Na for AAPL
Na for BA
Na for CAT
Na for CVX
Na for CSCO
Na for KO
Na for DOW
Na for GS
Na for HD
Na for HON
Na for IBM
Na for INTC
Na for JNJ
Na for JPM
Na for MCD
Na for MRK
buying signal for MSFT
Na for NKE
Na for PG
Na for CRM
Na for TRV
Na for UNH
Na for VZ
Na for V
Na for WBA
Na for WMT
Na for DIS


In [31]:
output[0]

,Date,Close,MACD_diff,Decision MACD,RSI,SMA200,Decision RSI/SMA
0,1986-03-13 00:00:00.000000,0.097222,NaN,NaN,NaN,NaN,NaN
1,1986-03-14 00:00:00.000000,0.100694,NaN,NaN,NaN,NaN,NaN
2,1986-03-17 00:00:00.000000,0.102431,NaN,NaN,NaN,NaN,NaN
3,1986-03-18 00:00:00.000000,0.099826,NaN,NaN,NaN,NaN,NaN
4,1986-03-19 00:00:00.000000,0.098090,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
8868,2021-05-19 00:00:00.000000,243.119995,-1.223848,NaN,41.008443,225.77580,NaN
8869,2021-05-20 00:00:00.000000,246.479996,-0.868503,NaN,47.822454,225.94175,NaN
8870,2021-05-21 00:00:00.000000,245.169998,-0.677226,NaN,45.543538,226.10290,NaN
8871,2021-05-24 00:00:00.000000,250.779999,-0.155648,NaN,55.609144,226.27505,NaN
